# Let's Prepare Dataset

## [0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral]

# Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import PIL
from PIL import Image
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm
# from tabulate import tabulate
import cv2

In [ ]:
num_classes=7

# Dataset

In [ ]:
class FERDataset(Dataset):
    def __init__(self, csv_data, transform, train=True):
        self.data = pd.read_csv(csv_data)
        self.transform = transform
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if self.train:
            pixels = self.data.iloc[idx, 1].split()
            pixels = np.array(pixels, dtype=np.uint8).reshape(48, 48)
    
            image = Image.fromarray(pixels)
    
            label = int(self.data.iloc[idx, 0])
    
            if self.transform:
                image = self.transform(image)
    
            return image, label

        pixels = self.data.iloc[idx, 0].split()
        pixels = np.array(pixels, dtype=np.uint8).reshape(48, 48)

        image = Image.fromarray(pixels)

        if self.transform:
            image = self.transform(image)

        return image


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=0.5, std=0.5)
])

# Function to generate All Dataloaders

In [ ]:
def generate_TrainValTest_dataloaders(trpath='', tstpath='', batch_size=32):
    Train_fer_dataset = FERDataset(csv_data=trpath, transform=transform)
    
    train_size = int(0.8 * len(Train_fer_dataset))
    val_size = len(Train_fer_dataset) - train_size

    Train_fer_dataset, Val_fer_dataset = random_split(Train_fer_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

    Test_fer_dataset = FERDataset(csv_data=tstpath, transform=transform, train=False)

    TrainDataLoader = DataLoader(Train_fer_dataset, batch_size=batch_size, shuffle=True)
    ValDataLoader = DataLoader(Val_fer_dataset, batch_size=batch_size)
    TestDataLoader = DataLoader(Test_fer_dataset, batch_size=batch_size)
    return TrainDataLoader, ValDataLoader, TestDataLoader

In [ ]:
train, val, test = generate_TrainValTest_dataloaders('data/train.csv', 'data/test.csv')

In [ ]:
len(train), len(val), len(test)

In [ ]:
emotion_dict = {
    0:'Angry',
    1:'Disgust',
    2:'Fear',
    3:'Happy',
    4:'Sad',
    5:'Surprise',
    6:'Neutral',
}

emotion_dict[1]

# Plot images

In [ ]:
train_features, train_labels = next(iter(train))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {emotion_dict[label.item()]}")

# Architecture

In [ ]:
# Model Architecture
class SimpleEmotionCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleEmotionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def evaluate(self, dataloader):
        self.eval()  # Set the model to evaluation mode
        y_true = []
        y_pred = []
    
        with torch.no_grad():
            for batch in tqdm(ValDataLoader, desc='Evaluating', leave=False):
                inputs, labels = batch
                outputs = self(inputs)
                predicted_labels = torch.argmax(outputs, dim=1)
    
                y_true.extend(labels.tolist())
                y_pred.extend(predicted_labels.tolist())
    
        accuracy = accuracy_score(y_true, y_pred)
        report = classification_report(y_true, y_pred, target_names=["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"], zero_division=1)
    
        return accuracy, report

    def predict(self, input_data, emotion_mapping):
        self.eval()
        with torch.no_grad():
            # Generate predictions based on input_data
            predictions = self(input_data)
        
            # Map numerical predictions to emotion labels
            predicted_labels = torch.argmax(predictions, dim=1)
            emotion_predictions = [emotion_mapping[label.item()] for label in predicted_labels]
    
            return emotion_predictions

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training Loop

In [ ]:
simpleCNN = SimpleEmotionCNN(num_classes)

# Define loss function and optimize
criterion = nn.CrossEntropyLoss(
optimizer = optim.SGD(simpleCNN.parameters(), lr=0.001, momentum=0.9)

num_epochs = 50

# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    simpleCNN.train()
    for batch in tqdm(TrainDataLoader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = simpleCNN(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    average_loss = total_loss / len(TrainDataLoader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {average_loss:.4f}")

# Save Model

In [ ]:
torch.save({
    'model_state_dict': simpleCNN.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch,
}, 'model_checkpoint_50.pth')

# Load Model

In [ ]:
checkpoint = torch.load('model_checkpoint.pth')

simpleCNN = SimpleEmotionCNN(num_classes)

simpleCNN.load_state_dict(checkpoint['model_state_dict'])

epoch = checkpoint['epoch']

# Evaluation

In [ ]:
accuracy, report = simpleCNN.evaluate(ValDataLoader)

### Model's Accuracy

In [ ]:
accuracy

### Classification Report

In [ ]:
report_lines = report.split('\n')
# Print each line with proper formatting
for line in report_lines:
    print(line)

# Preprocessing Image for Prediction

In [ ]:
def preprocess_image(image_path, target_size=(48, 48)):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (48, 48))

    image_tensor = torch.Tensor([image])
    image_tensor = image_tensor.view(1, 1, 48, 48)

    return image_tensor

In [ ]:
image_path = 'F:\Class-BSSE\pp.jpeg'
image_path1 = 'F:\Class-BSSE\IMG_20210513_073359.jpg'
image_path2 = 'F:\Class-BSSE\IMG_20210513_073354.jpg'
image_path3 = 'myface.PNG'
image_path4 = 'myface1.PNG'
image_path4 = 'shaggy.PNG'

In [ ]:
preprocessed_image = preprocess_image(image_path)

image_array = preprocessed_image.squeeze().numpy()
plt.imshow(image_array, cmap='gray')
plt.show()

In [ ]:
emotion_mapping = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprise',
    6: 'Neutral'
}

# Prediction

In [ ]:
emotion = simpleCNN.predict(preprocessed_image, emotion_mapping)
emotion

# Testing

In [ ]:
def plot_images_with_predictions(images, predictions, class_names):
    num_images = len(images)
    
    # Create a grid to display images and predictions
    num_cols = 4  # You can adjust the number of columns as needed
    num_rows = (num_images + num_cols - 1) // num_cols

    
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 3*num_rows))
    
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            image = images[i].numpy().squeeze()
            predicted_probs = predictions[i].cpu().numpy()
            top3_indices = np.argsort(predicted_probs)[::-1][:3]
            
            ax.imshow(image, cmap='gray')
            ax.set_title('Top 3 Predictions:')
            for j, idx in enumerate(top3_indices):
                ax.set_title(f'{ax.get_title()}\n{j+1}: {class_names[idx]} ({predicted_probs[idx]:.2f})')
            ax.axis('off')
        else:
            ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Testing loop with image plotting
simpleCNN.eval()  # Set the model to evaluation mode
class_names = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

with torch.no_grad():
    for batch in TestDataLoader:
        outputs = simpleCNN(batch)  # Assuming batch contains only images

        # Plot images with top 3 predictions for this batch
        plot_images_with_predictions(batch, outputs, class_names)
        
        # Optionally, ask if the user wants to see the next batch
        next_batch = input("Show the next batch of images? (y/n): ")
        if next_batch.lower() != 'y':
            break  # Exit the loop if the user doesn't want to see the next batch

